## Spoken GAN

### A Generative Adversarial Network (GAN) for Spoken Digits

In [ ]:
# SOURCE SPOKEN DIGIT DATASET

!git clone https://github.com/Jakobovski/free-spoken-digit-dataset.git

In [2]:
# GET FILE NAMES OF SAMPLES

from os import listdir, getcwd
from os.path import isfile, join

data_dir = join(getcwd(), 'free-spoken-digit-dataset', 'recordings')

file_names = [f for f in listdir(data_dir) if f[-4:] == '.wav']

In [3]:
# IMPORT DATA, PAD IT, ARRANGE INTO TENSOR

from scipy.io import wavfile
import numpy as np

wavs = []
for f in file_names:
    _, wav = wavfile.read(join(data_dir, f))
    wavs.append(wav)
    
max_length = max([w.shape[0] for w in wavs])
# padded_length is least multiple of 128 greater
# than 512 containing max_length
padded_length = -128 * (-max_length // 128)


padded_wavs = []
for wav in wavs:
    pad_size = padded_length - wav.shape[0]
    left_pad = pad_size // 2
    right_pad = pad_size - left_pad
    padded_wavs.append(np.pad(wav, (left_pad, right_pad), mode='constant'))
    
X = np.stack(padded_wavs, axis=0)
Y = np.array([[int(f[:1]),] for f in file_names])

In [4]:
import tensorflow as tf
from tensorflow.keras import layers

BUFFER_SIZE = X.shape[0]
BATCH_SIZE = 128

train_dataset = tf.data.Dataset.from_tensor_slices(X).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [5]:
from IPython.display import Audio
def speak(sample):
    return Audio(sample, rate=8000, autoplay=True)

In [6]:
class InverseSTFTLayer(tf.keras.layers.Layer):
    def __init__(self, frame_length, frame_step):
        super(InverseSTFTLayer, self).__init__()
        self.frame_length = frame_length
        self.frame_step = frame_step
        #self.num_frames = 1 + (num_outputs - frame_length) // frame_step
        #self.input_shape = (self.num_frames, 1 + frame_length // 2)


    def build(self, input_shape):
        self.num_frames = input_shape[-2]
        self.num_outputs = self.frame_length + (self.num_frames - 1) * self.frame_step

    def call(self, input):
        inverse_stft = tf.signal.inverse_stft(
            input, self.frame_length, self.frame_step,
            window_fn=tf.signal.inverse_stft_window_fn(self.frame_step)
        )
        return inverse_stft

In [7]:
class ComplexLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(ComplexLayer, self).__init__()

    def build(self, input_shape):
        pass

    def call(self, input):
        real = input[...,0]
        imag = input[...,1]
        return tf.complex(real, imag)

In [8]:
frame_length = 512
frame_step = 128

l = InverseSTFTLayer(frame_length, frame_step)

waveform = tf.random.normal(dtype=tf.float32, shape=[padded_length])
stft = tf.signal.stft(waveform, frame_length, frame_step, pad_end=False)
output = l(stft)


In [9]:
stft.shape

TensorShape([140, 257])

In [10]:
frame_length = 512
frame_step = 128
waveform = tf.random.normal(dtype=tf.float32, shape=[4096])
stft = tf.signal.stft(waveform, frame_length, frame_step, pad_end=False)
inverse_stft = tf.signal.inverse_stft(
    stft, frame_length, frame_step,
    window_fn=tf.signal.inverse_stft_window_fn(frame_step)
)
output = tf.where(tf.math.is_nan(inverse_stft), tf.zeros_like(inverse_stft), inverse_stft)

speak(waveform - output)

In [11]:
# speak(tf.pad(inverse_stft, tf.constant([[0,1024 - inverse_stft.shape[0]]]))[1:] - waveform[1:])

In [12]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(10*10*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((10, 10, 256)))
    assert model.output_shape == (None, 10, 10, 256) # Note: None is the batch size
    
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(3,4), padding='valid', use_bias=False))
    assert model.output_shape == (None, 32, 41, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (6,5), strides=(2,2), padding='valid', use_bias=False))
    assert model.output_shape == (None, 68, 85, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(2, (6,5), strides=(2,3), padding='valid', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 140, 257, 2)
    
    model.add(ComplexLayer())
    model.add(InverseSTFTLayer(512, 128))
    
    return model

In [15]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_speech = generator(noise, training=False)

speak(generated_speech)

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[28, 28, 2]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model